verifying the original and python implementation of cai/bai

In [15]:
import os
cwd = os.getcwd()

pygad_loc = '/grid/home/nbourgeois/hmi_bifx_tools/codon_optimization'
os.chdir(pygad_loc)
from metrics import get_bai as get_bai_old
from metrics import get_cai as get_cai_old


In [17]:
def get_codon_weights(index_loc):
    
    codon_table_raw = pd.read_csv(index_loc,sep='\t')
    codon_table = codon_table_raw.set_index('#Codon')
    maxes = codon_table_raw.groupby(by=['AA'])['Frequency'].max()
    codon_table['max_col'] = codon_table.apply(lambda x: maxes[x['AA']],axis=1)
    codon_table['weight'] = codon_table['Frequency'] / codon_table['max_col']
    weight_dict = codon_table['weight'].to_dict()
    
    return(weight_dict)

def get_cai(seq, weight_dict):
    if weight_dict is str:
        weight_dict = get_codon_weights(weight_dict)
    weights = [weight_dict[seq[i:i+3]] for i in range(0, len(seq)-3, 3)] #convert to codon_list -> use weight dictionary
    cai = geo_mean(weights)
    return(cai)

def geo_mean(iterable):
    a = np.array(iterable)
    a = np.log(a)
    return np.exp(a.sum() / len(a))

def get_bicodon_weights(index_loc):
    
    def bicodon_to_AA(string):
        cod1 = string[:3]
        cod2 = string[3:]
        biAA = forward_table[cod1] + forward_table[cod2]
        return(biAA)
    
    def get_forward_table():
        codon_usage_table_loc = '~/references/codon_usage.getex.txt'
        codon_usage_table = pd.read_csv(codon_usage_table_loc,sep='\t')
        forward_table = pd.Series(codon_usage_table.AA.values,index=codon_usage_table.Codon).to_dict()
        return(forward_table)
     
    # create table
    codon_table_raw_t = pd.read_csv(index_loc,sep='\t')
    codon_table = codon_table_raw_t.T

    # add amino acid pair column
    bicodons_nt = list(codon_table.index)
    forward_table = get_forward_table()
    codon_table['AApair'] = list(map(bicodon_to_AA,bicodons_nt))
    total = codon_table[0].sum()
    
    #add frequency
    bicodons_nt = list(codon_table.index)
    codon_table['AApair'] = list(map(bicodon_to_AA,bicodons_nt))
    total = codon_table[0].sum()
    codon_table['Frequency'] = codon_table.apply(lambda x: (x[0] / total * 1000) ,axis=1)
    
    #calculate weights
    maxes = codon_table.groupby(by=['AApair'])['Frequency'].max()
    codon_table['max_col'] = codon_table.apply(lambda x: maxes[x['AApair']],axis=1)
    codon_table['weight'] = codon_table['Frequency'] / codon_table['max_col']
    
    weight_dict = codon_table['weight'].to_dict()
    
    return(weight_dict)

def get_bai(seq, weight_dict):
    if weight_dict is str:
        weight_dict = get_bicodon_weights(weight_dict)
        
    # sliding window bicodon, excludes the last codon
    weights = [weight_dict[seq[i:i+6]] for i in range(0, len(seq)-6, 3)] #convert to codon_list -> use weight dictionary
    
    bai = geo_mean(weights)
    return(bai)

In [9]:
dirs = ['/grid/home/nbourgeois/data/test_sequences/pah',
        '/grid/home/nbourgeois/data/test_sequences/hidua']

fastaIDs, seqs = readFasta('/grid/home/nbourgeois/data/test_sequences/PAH_codon_random_10k.fa')
        

In [18]:
cai_index_loc = os.path.join(pygad_loc,'CoCoPUTs_codon_usage/codon_usage/'+'Liver'+'.codon.txt')
bai_index_loc = os.path.join(pygad_loc,'CoCoPUTs_codon_usage/bicodon_usage/'+'Liver'+'.bicodon.txt')

codon_usage_table_loc = os.path.join(pygad_loc,'references','codon_usage.getex.txt')
cai_weight_dict = get_codon_weights(cai_index_loc)
bai_weight_dict = get_bicodon_weights(bai_index_loc)

In [29]:
get_cai(seqs[1],cai_weight_dict)
# get_cai_old(seq,cai_index_loc)

round(get_cai_old(seq,cai_index_loc),3)

0.633

In [30]:
round(get_cai_old(seq,cai_index_loc),3) - round(get_cai(seq,cai_weight_dict),3)

0.0010000000000000009

In [36]:
x=10


True

In [37]:
i=0
for seq in seqs:
    if i % 100 == 0:
        print(i)
    seq = str(seq)
    
    cai = round(get_cai(seq,cai_weight_dict),3)
    old_cai = round(get_cai_old(seq,cai_index_loc),3)
    if cai - old_cai > .001 :
        print('Mismatch: CAI',cai,old_cai)
        
    
    bai = round(get_bai(seq,bai_weight_dict),6)
    old_bai = round(get_bai_old(seq,bai_index_loc),6)
    if bai - old_bai > .00001 :
        print('Mismatch: BAI',bai,old_bai)
        
    i +=1
    
    
    

0
Mismatch: CAI 0.657 0.656
100
Mismatch: CAI 0.658 0.657
200
Mismatch: CAI 0.642 0.641
300
400
Mismatch: CAI 0.64 0.639
500
Mismatch: CAI 0.661 0.66
Mismatch: CAI 0.638 0.637
Mismatch: CAI 0.64 0.639
600
700
Mismatch: CAI 0.645 0.644
800
Mismatch: CAI 0.633 0.632
900
1000
Mismatch: CAI 0.644 0.643
1100
Mismatch: CAI 0.659 0.658
1200
Mismatch: CAI 0.655 0.654
1300
Mismatch: CAI 0.66 0.659
1400
Mismatch: CAI 0.638 0.637
1500
1600
Mismatch: CAI 0.655 0.654
1700
1800
1900
Mismatch: CAI 0.667 0.666
Mismatch: CAI 0.643 0.642
Mismatch: CAI 0.644 0.643
Mismatch: CAI 0.639 0.638
2000
2100
Mismatch: CAI 0.648 0.647
2200
Mismatch: CAI 0.668 0.667
2300
2400
Mismatch: CAI 0.658 0.657
Mismatch: CAI 0.661 0.66
Mismatch: CAI 0.64 0.639
2500
2600
Mismatch: CAI 0.651 0.65
2700
Mismatch: CAI 0.652 0.651
Mismatch: CAI 0.665 0.664
2800
Mismatch: CAI 0.646 0.645
2900
3000
Mismatch: CAI 0.647 0.646
Mismatch: CAI 0.652 0.651
3100
Mismatch: CAI 0.655 0.654
3200
Mismatch: CAI 0.64 0.639
Mismatch: CAI 0.655 0.6

In [23]:
round(.0123456789,5) ==round(.01235005646,5)

True

0.01236